# Overlapとmp_term_nameの統合

- 例：abnormal_circulating_glucose_level（連続値）
- 例：male_infertility（カテゴリ値）

In [2]:
# Move up to top directory
import os
os.getcwd()
os.chdir('../')
os.getcwd()

'/mnt/g/impc-genes_with_similar_phenotypes'

In [3]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import json

In [4]:
# path_overlap = Path("data", "overlap_ratios_filtered.csv")
# df_overlap = pd.read_csv(path_overlap)

overlap = pickle.load(open("data/overlapeed_ratios_filtered.pkl", "rb"))
df_overlap = pd.DataFrame(overlap, columns=["marker1", "marker2", "overlap_ratio", "overlapped_mp_number", "overlapped_mp"])
print(df_overlap)

       marker1  marker2  overlap_ratio  overlapped_mp_number  \
0         Dpf2     Aff4          0.200                     4   
1         Dpf2    Fgf15          0.333                     4   
2         Dpf2  Col11a1          0.231                     3   
3         Dpf2   Sptan1          0.154                     4   
4         Dpf2      Nog          0.160                     4   
...        ...      ...            ...                   ...   
128944    Mcm4   Exosc8          1.000                     3   
128945  Zfp407   Washc4          0.750                     3   
128946    Gmnn    Rab35          1.000                     3   
128947   Pola2   Exosc8          1.000                     3   
128948    Mrm2      Jtb          0.600                     3   

                                            overlapped_mp  
0       [abnormal craniofacial morphology, abnormal ki...  
1       [abnormal embryo size, abnormal heart morpholo...  
2       [abnormal limb morphology, abnormal tail mo

In [33]:
# path_marker_effect = Path("data", "mp_term_name", "decreased_circulating_glucose_level.csv")
path_marker_effect = Path("data", "mp_term_name", "increased_circulating_glucose_level.csv")
df_marker_effect = pd.read_csv(path_marker_effect)
df_marker_effect = df_marker_effect[["marker_symbol", "effect_size"]].drop_duplicates()


In [36]:
print(len(df_marker_effect))

df_marker_effect.head(3)


49


,marker_symbol,effect_size
0,Zfp513,0.200596
1,Dbnl,0.828539
2,Oas1g,0.922229


In [37]:
# print(df_mp["effect_size"].describe())

In [38]:
# Absolute value of effect size

df_marker_effect.loc[:, "effect_size"] = df_marker_effect["effect_size"].abs()

In [39]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [40]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.25)]

In [41]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_marker_effect['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_marker_effect['marker_symbol']))
]

In [42]:

print(len(df_filtered))

df_filtered.head(3)

10


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp
1035,Zfyve19,Ky,0.333,3,"[abnormal behavior, decreased exploration in n..."
8493,Bbs5,Bscl2,0.262,11,"[abnormal bone structure, abnormal gait, decre..."
63049,Mrpl39,Znhit2,0.500,3,"[decreased circulating bilirubin level, increa..."


### 出力

In [43]:
# Edge list

df_edge = df_filtered[["marker1", "marker2", "overlap_ratio", "overlapped_mp"]]
print(len(df_edge))
print(df_edge.head(3))


10
       marker1 marker2  overlap_ratio  \
1035   Zfyve19      Ky          0.333   
8493      Bbs5   Bscl2          0.262   
63049   Mrpl39  Znhit2          0.500   

                                           overlapped_mp  
1035   [abnormal behavior, decreased exploration in n...  
8493   [abnormal bone structure, abnormal gait, decre...  
63049  [decreased circulating bilirubin level, increa...  


In [44]:
# JSON形式に変換
edge_json = []
for _, row in df_edge.iterrows():
    edge_json.append({
        "data": {
            "source": row['marker1'],
            "target": row['marker2'],
            "annotation": row['overlapped_mp'],
            "edge_size": row['overlap_ratio']
        }
    })

print(edge_json[:3])

[{'data': {'source': 'Zfyve19', 'target': 'Ky', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'increased circulating glucose level'], 'edge_size': 0.333}}, {'data': {'source': 'Bbs5', 'target': 'Bscl2', 'annotation': ['abnormal bone structure', 'abnormal gait', 'decreased bone mineral content', 'decreased respiratory quotient', 'increased circulating alanine transaminase level', 'increased circulating alkaline phosphatase level', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased mean corpuscular volume', 'increased monocyte cell number', 'short tibia'], 'edge_size': 0.262}}, {'data': {'source': 'Mrpl39', 'target': 'Znhit2', 'annotation': ['decreased circulating bilirubin level', 'increased circulating glucose level', 'preweaning lethality, complete penetrance'], 'edge_size': 0.5}}]


In [45]:
# Nodeの情報を作成
# merge mp term
df_marker1 = df_filtered[["marker1"]]
df_marker2 = df_filtered[["marker2"]]
df_node_marker1 = pd.merge(df_marker1, df_marker_effect, left_on='marker1', right_on='marker_symbol', how='inner')[["marker_symbol"]]
df_node_marker2 = pd.merge(df_marker2, df_marker_effect, left_on='marker2', right_on='marker_symbol', how='inner')[["marker_symbol"]]

df_node = pd.concat([df_node_marker1, df_node_marker2], axis=0).drop_duplicates()

print(df_node)
import pickle
marker_mp = pickle.load(open("data/marker_mp.pkl", "rb"))
marker_mp = pd.DataFrame(marker_mp.items(), columns=["marker_symbol", "mp_term"])

df_node = pd.merge(df_node, marker_mp, how='inner', on='marker_symbol')
print(df_node)

df_node = pd.merge(df_node, df_marker_effect, how='inner', on='marker_symbol')

  marker_symbol
0       Zfyve19
1          Bbs5
2        Mrpl39
5        Dtnbp1
6         Eid2b
7        Znhit2
9          Gcn1
0            Ky
1         Bscl2
4        Thap12
5      Slc25a46
   marker_symbol                                            mp_term
0        Zfyve19  [abnormal behavior, decreased exploration in n...
1           Bbs5  [abnormal bone structure, abnormal gait, abnor...
2         Mrpl39  [decreased circulating bilirubin level, decrea...
3         Dtnbp1  [abnormal coat/hair pigmentation, abnormal iri...
4          Eid2b  [decreased mean corpuscular volume, embryonic ...
5         Znhit2  [decreased circulating bilirubin level, embryo...
6           Gcn1  [decreased circulating bilirubin level, increa...
7             Ky  [abnormal behavior, abnormal gait, decreased e...
8          Bscl2  [abnormal bone structure, abnormal gait, decre...
9         Thap12  [decreased circulating bilirubin level, increa...
10      Slc25a46  [decreased grip strength, increased circul

In [46]:
# JSON形式に変換
node_json = []
for _, row in df_node.iterrows():
    node_json.append({
        "data": {
            "id": row['marker_symbol'],
            "label": row['marker_symbol'],
            "annotation": row['mp_term'],
            "node_color": row['effect_size']
        }
    })

print(node_json[:3])

[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annotation': ['abnormal bone structure', 'abnormal gait', 'abnormal retina morphology', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased circulating triglyceride level', 'decreased grip strength', 'decreased heart weight', 'decreased lean body mass', 'decreased respiratory quotient', 'impaired glucose tolerance', 'increased circulating alanine transaminase level', 'increased circulating alkaline phosphatase level', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased hematocrit', 'increased hemoglobin content', 'increased leukocyte cell number', 'increased lymphocyte cell number', 'increased mean corpuscular volume', 'increased monocyte cell

In [47]:
# Combine node and edge

network_json = node_json + edge_json

In [48]:
print(len(network_json))
print(network_json[:3])
print(network_json[-3:])

21
[{'data': {'id': 'Zfyve19', 'label': 'Zfyve19', 'annotation': ['abnormal behavior', 'decreased exploration in new environment', 'decreased thigmotaxis', 'increased circulating glucose level'], 'node_color': 0.695783774710263}}, {'data': {'id': 'Bbs5', 'label': 'Bbs5', 'annotation': ['abnormal bone structure', 'abnormal gait', 'abnormal retina morphology', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased circulating triglyceride level', 'decreased grip strength', 'decreased heart weight', 'decreased lean body mass', 'decreased respiratory quotient', 'impaired glucose tolerance', 'increased circulating alanine transaminase level', 'increased circulating alkaline phosphatase level', 'increased circulating aspartate transaminase level', 'increased circulating glucose level', 'increased hematocrit', 'increased hemoglobin content', 'increased leukocyte cell number', 'increased lymphocyte cell number', 'increased mean corpuscular volume', 'increased monocyte c

In [49]:
# Output as JSON file

with open("data/network.json", "w") as f:
    json.dump(network_json, f, indent=2)

# Debug

In [22]:
for data in node_json:
    if "Lepr" == data["data"]["id"]:
        print(data)

{'data': {'id': 'Lepr', 'label': 'Lepr', 'annotation': ['abnormal bone structure', 'abnormal gait', 'abnormal locomotor behavior', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased circulating chloride level', 'decreased exploration in new environment', 'decreased heart weight', 'decreased lean body mass', 'decreased leukocyte cell number', 'decreased locomotor activity', 'decreased lymphocyte cell number', 'decreased spleen weight', 'enlarged liver', 'female infertility', 'impaired righting response', 'increased anxiety-related response', 'increased circulating HDL cholesterol level', 'increased circulating alanine transaminase level', 'increased circulating alkaline phosphatase level', 'increased circulating aspartate transaminase level', 'increased circulating calcium level', 'increased circulating cholesterol level', 'increased circulating fructosamine level', 'increased circulating iron level', 'increased circulating phosphate level', 'increased circul

In [163]:
for data in edge_json:
    if "Herc1" == data["data"]["source"]:
        print(data)
    if "Herc1" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Camsap3', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnormal lens morphology', 'cataract', 'decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.31}}
{'data': {'source': 'Prdm14', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'impaired glucose tolerance', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.273}}
{'data': {'source': 'Kash5', 'target': 'Herc1', 'annotation': ['decreased bone mineral content', 'decreased bone mineral density', 'decreased lean body mass', 'increased leukocyte cell number', 'increased total body fat amount', 'male infertility'], 'edge_size': 0.286}}
{'data': {'source': 'Trim37', 'target': 'Herc1', 'annotation': ['abnormal auditory brainstem response', 'abnor

In [150]:
for data in edge_json:
    if "Stoml2" == data["data"]["source"]:
        print(data)
    if "Stoml2" == data["data"]["target"]:
        print(data)

{'data': {'source': 'Vgf', 'target': 'Stoml2', 'annotation': ['decreased prepulse inhibition', 'increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.25}}
{'data': {'source': 'Fbxo38', 'target': 'Stoml2', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.214}}
{'data': {'source': 'Stoml2', 'target': 'Clcn1', 'annotation': ['decreased prepulse inhibition', 'increased circulating aspartate transaminase level', 'increased circulating bilirubin level', 'male infertility'], 'edge_size': 0.182}}
{'data': {'source': 'Stoml2', 'target': 'Lepr', 'annotation': ['increased circulating alanine transaminase level', 'increased circulating aspartate transaminase level', 'male infertility'], 'edge_size': 0.073}}
{'data': {'source': 'Stoml2', 'target': 'Atp13a3', 'annotation': ['increased circulating alanine tra